In [1]:
from pwn import *
import random

def recv():
    outputs = []
    ciphertext = None

    for i in range(0, 624, 8):
        # f = process(["python3", "chall.py"])
        f = connect("vsc.tf", 5001)
        context.log_level = 'debug'
        
        indices = list(range(i, min(i + 8, 624)))
        f.sendlineafter(b">>>", str(indices).encode())
        
        for _ in indices:
            try:
                output = int(f.recvline().strip())
                outputs.append(output)
            except EOFError:
                break

        if ciphertext is None:
            try:
                ciphertext = f.recvline().strip().decode()
                ciphertext = bytes.fromhex(ciphertext)
            except EOFError:
                break
        
        f.close()
    
    return outputs, ciphertext

outputs, ciphertext = recv()
print(outputs)
print(ciphertext)
print(len(ciphertext))

from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from hashlib import sha256

import randcrack

rc = randcrack.RandCrack()

for output in outputs:
    rc.submit(output)

key = rc.predict_getrandbits(256)
nonce = rc.predict_getrandbits(256)
print(key)
print(nonce)


cipher = AES.new(sha256(str(key).encode()).digest()[:16], AES.MODE_GCM, nonce=sha256(str(nonce).encode()).digest()[:16])
print(cipher.decrypt(ciphertext))

[x] Opening connection to vsc.tf on port 5001
[x] Opening connection to vsc.tf on port 5001: Trying 34.46.154.169
[+] Opening connection to vsc.tf on port 5001: Done
[DEBUG] Received 0x4 bytes:
    b'>>> '
[DEBUG] Sent 0x19 bytes:
    b'[0, 1, 2, 3, 4, 5, 6, 7]\n'
[DEBUG] Received 0xb7 bytes:
    b'475736381\n'
    b'2599178730\n'
    b'639420308\n'
    b'1400092731\n'
    b'2055728736\n'
    b'3311659743\n'
    b'3349287473\n'
    b'3292155355\n'
    b'cb3cc14d2f5eeac6b5645bb66fa268d88399d1654df20668110e1d04bf135db71930985b5eba307c0197b035f2e9203f\n'
[*] Closed connection to vsc.tf port 5001
[x] Opening connection to vsc.tf on port 5001
[x] Opening connection to vsc.tf on port 5001: Trying 34.46.154.169
[+] Opening connection to vsc.tf on port 5001: Done
[DEBUG] Received 0x4 bytes:
    b'>>> '
[DEBUG] Sent 0x1f bytes:
    b'[8, 9, 10, 11, 12, 13, 14, 15]\n'
[DEBUG] Received 0xb7 bytes:
    b'3972270152\n'
    b'3453764163\n'
    b'774805574\n'
    b'1416637968\n'
    b'1686108321\n'
 